In [ ]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
import os
import pickle
import pathlib

from naludaq.board import Board, startup_board
from naludaq.communication import AnalogRegisters, DigitalRegisters, ControlRegisters
from naludaq.parsers import get_parser
from naludaq.tools import get_pedestals_controller, get_threshold_scanner
from naludaq.tools.pedestals.pedestals_correcter import PedestalsCorrecter

from naludaq.backend import AcquisitionManager, DiskAcquisition
from naludaq.controllers import (
    get_board_controller,
    get_dac_controller,
    get_readout_controller,
)

import matplotlib.pyplot as plt

### Directory Setup

In [ ]:
DATA_DIR = pathlib.Path().cwd() / "data"
FIGURE_DIR = pathlib.Path().cwd() / "figures"
CALIBRATIONS_DIR = pathlib.Path().cwd() / "calibrations"

### Function Definitions

In [ ]:
def create_acquisition(name: str, board: Board, readout_metadata: dict = None):
    am = AcquisitionManager(board)
    acquisition = am.create(name)
    acquisition.set_output()
    if readout_metadata:
        acquisition.readout_metadata = readout_metadata
    return acquisition

def create_acq_name(base_name: str):
    timestr = time.strftime("%Y-%m-%d-%H-%M-%S")
    return base_name + "_" + timestr

def parse_data(raw_data: dict, board: Board, correct_peds: bool = True):
    PARSER = get_parser(board.params)
    evt = PARSER.parse(raw_data)
    if correct_peds:
        PC = PedestalsCorrecter(board.params, board.pedestals)
        PC.run(evt)
    return evt

def calculate_max_trigger(threshold_data):
    trigger_values = []
    for threshold_hits in threshold_data[0]:
        max_threshold = threshold_data[1][np.argmax(threshold_hits)]
        trigger_values.append(max_threshold)
    return trigger_values

def clear_figs():
    plt.close("all")

def create_fig(figsize: tuple):
    fig = plt.figure(figsize=figsize, constrained_layout=True)
    return fig

def plot_single_event(event, show: bool = True):
    for chan, chan_data in enumerate(event["data"]):
        if isinstance(chan_data, np.ndarray) and chan_data.size == 0:
            continue
        if len(chan_data) == 0:
            continue
        ax = plt.plot(chan_data, '.-', markersize=10, label=f"Channel {chan}")
    plt.legend(fontsize=12)
    if show:
        plt.show()


### Startup Board

In [ ]:
BOARD = Board('hdsocv1_evalr2')
BOARD.start_server("data")
BOARD_ADDRESS = ("192.168.1.59", 4660)
RECEIVER_ADDRESS = ("192.168.1.165", 4662)
SERVER_ADDRESS = (BOARD.context.address)
BOARD.connect_udp(BOARD_ADDRESS, RECEIVER_ADDRESS, SERVER_ADDRESS)
startup_board(BOARD)

### Function Gen Setup

In [ ]:
from naluinstruments.Instruments import Siglent_FuncGen
import time

CHANNEL_INPUT = Siglent_FuncGen(1, "192.168.1.189")

PULSE_FREQ = 20
PULSE_WIDTH = 10e-9
PULSE_AMP = 0.15

CHANNEL_INPUT.rf_off()

# REGULAR PULSE
CHANNEL_INPUT.set_burst("OFF")
CHANNEL_INPUT.set_wavetype("PULSE")
CHANNEL_INPUT.set_width(PULSE_WIDTH)
CHANNEL_INPUT.set_rise(2e-9)
CHANNEL_INPUT.set_amp(PULSE_AMP)
CHANNEL_INPUT.set_freq(PULSE_FREQ)

# BURST PULSE
# NUM_PULSES = 10
# PULSE_FREQ = 50e6
# BURST_PERIOD = 1
# CHANNEL_INPUT.set_wavetype("PULSE")
# CHANNEL_INPUT.set_width(PULSE_WIDTH)
# CHANNEL_INPUT.set_rise(2e-9)
# CHANNEL_INPUT.set_amp(PULSE_AMP)
# CHANNEL_INPUT.set_freq(PULSE_FREQ)

# CHANNEL_INPUT.set_burst("ON")
# CHANNEL_INPUT.set_burst_mode("NCYC")
# CHANNEL_INPUT.set_burst_time(NUM_PULSES)
# CHANNEL_INPUT.set_burst_period(BURST_PERIOD)

print(CHANNEL_INPUT.get_output_settings())

### Set DACS

In [ ]:
get_dac_controller(BOARD).set_dacs(1600)

### Generate / Load Pedestals

In [ ]:
USE_OLD_PEDS = True

if USE_OLD_PEDS and os.path.exists(CALIBRATIONS_DIR / "hdsocv1_evalr2.peds"):
    with open(CALIBRATIONS_DIR / "hdsocv1_evalr2.peds", "rb") as file:
        peds = pickle.load(file)
    BOARD.pedestals = peds
else:
    PC = get_pedestals_controller(BOARD)
    PC.generate_pedestals()
    with open(CALIBRATIONS_DIR / "hdsocv1_evalr2.peds", "wb") as file:
        pickle.dump(BOARD.pedestals, file)

### Calculate Triggers Thresholds

In [ ]:
USE_OLD_THRESHOLDS = True
STEP_SIZE = 5
LOW_REF = 6
HIGH_REF = 12
THRESH_START = BOARD.params["threshold_scan"]["start"]
THRESH_END = BOARD.params["threshold_scan"]["stop"]

if USE_OLD_THRESHOLDS and os.path.exists(CALIBRATIONS_DIR / "threshold_sweep.pkl"):
    with open(CALIBRATIONS_DIR / "threshold_sweep.pkl", "rb") as file:
        threshold_data = pickle.load(file)
else:
    AR = AnalogRegisters(BOARD)
    AR.write("tsel_left", 1)
    AR.write("tsel_right", 1)
    AR.write("tsgn_left", 1)
    AR.write("tsgn_right", 1)

    scanrange = np.arange(THRESH_START, THRESH_END, STEP_SIZE)
    threshold_scanner = get_threshold_scanner(BOARD)
    threshold_scanner.scan_values = scanrange
    threshold_scanner.low_ref_value = LOW_REF
    threshold_scanner.high_ref_value = HIGH_REF

    threshold_data = threshold_scanner.run()

    with open(CALIBRATIONS_DIR / "threshold_sweep.pkl", "wb") as file:
        pickle.dump(threshold_data, file)

### Plot Threshold Scan

In [ ]:
fig = create_fig(figsize=(10,8))
plt.xlabel("Trigger Level (DAC Counts)")
plt.ylabel("Occurences")
plt.title(f"HDSoCv1 Threshold Scan")
threshold_values = threshold_data[1]
for channel, threshold_hits in enumerate(threshold_data[0]):
    plt.plot(
        threshold_values, threshold_hits, ".-", ms=8, label=f"Channel {channel}"
    )
plt.legend(fontsize=10, loc="upper left")
plt.grid()

### Setup Triggers

In [ ]:
from naludaq.controllers.trigger import get_trigger_controller

TRIGGER_OFFSET = 20
CHANNELS_TO_SET = [1] # range(BOARD.channels)

DR = DigitalRegisters(BOARD)
AR = AnalogRegisters(BOARD)
TC = get_trigger_controller(BOARD)

TC.update_reference_voltages("left", LOW_REF, HIGH_REF)
TC.update_reference_voltages("right", LOW_REF, HIGH_REF)
AR.write("tsel_left", 1)
AR.write("tsel_right", 1)
AR.write("tsgn_left", 1)
AR.write("tsgn_right", 1)

trigger_values = [1] * BOARD.channels
max_trigger = BOARD.params["trigger"]["max_counts"]

# Calculate the trigger levels for each channel
max_trigger_values = calculate_max_trigger(threshold_data)
for chan in CHANNELS_TO_SET:
    trigger_values[chan] = min(int(max_trigger_values[chan] + TRIGGER_OFFSET), max_trigger)

BOARD.trigger_values = trigger_values
TC.write_triggers()

### Run ROI Mode Capture

In [ ]:
BC = get_board_controller(BOARD)
RC = get_readout_controller(BOARD)

# Set Readout Channels
RC.set_readout_channels([*range(BOARD.channels)])


WINDOW = 62
LOOKBACK = 62
WAT = 31
READOUTCHANNELS = 59 # Fine tune placement of pulse in event

TRIGGER_TIME = 5
READOUT_TIME = 0

RC.set_read_window(WINDOW, LOOKBACK, WAT)
DR.write("readoutchannels", READOUTCHANNELS)

READOUT_PARAMS = {
    "trig": "self",
    "lb": "roi",
    "acq": "raw",
    "ped": "zero",
    "readoutEn": True,
    "singleEv": False,
}

acq_name = create_acq_name("hdsocv1_evalr2_ROI")
acq = create_acquisition(acq_name, BOARD)

CHANNEL_INPUT.rf_on()
time.sleep(0.25)
try:
    BC.start_readout(**READOUT_PARAMS)
    time.sleep(TRIGGER_TIME)
finally:
    CHANNEL_INPUT.rf_off()
    time.sleep(READOUT_TIME)
    BC.stop_readout()
    time.sleep(0.25)

print(f"Num of events received: {len(acq)}")

### Parse Events

In [ ]:
events = []
for raw_event in acq:
    events.append(parse_data(raw_event, BOARD))

### Plot Single Event

In [ ]:
EVENT_IDX = 5
event = events[EVENT_IDX]

fig = create_fig(figsize=(10,8))
fig.set_facecolor('white')
plot_single_event(event, show=False)
plt.grid(True, axis="x")
plt.xlim([0, len(event["data"][1])])
plt.xticks([*range(0, len(event["data"][1]), 32)], labels=event["window_labels"][1])
plt.xlabel("Window Labels")
plt.title(f"WIN: {WINDOW}, LB: {LOOKBACK}, WAT: {WAT}, READOUT: {READOUTCHANNELS}, Pulse Width: {PULSE_WIDTH}, , Event: {EVENT_IDX}")

### Plot From Existing Acquisition

In [ ]:
AM = AcquisitionManager(BOARD)
ACQUISITIONS = AM.list()

ACQ = ACQUISITIONS[-1]
print(ACQ.name)

disk_acq = DiskAcquisition(str(DATA_DIR / ACQ.name))

evt = parse_data(disk_acq[1], BOARD)
fig = create_fig((10, 6))
fig.set_facecolor('white')
plot_single_event(evt, show=False)
plt.xlim([0, len(evt["data"][1])])
plt.xticks([*range(0, len(evt["data"][1]), 32)], labels=evt["window_labels"][1])
plt.xlabel("Window Labels")
plt.title(f"Pulse Width: {PULSE_WIDTH}")
    

### Using Data Collector

In [ ]:
from naludaq.tools.data_collector import get_data_collector
DC = get_data_collector(BOARD)
READ_WINDOW = {
    "windows": 62,
    "lookback": 62,
    "write_after_trig": 31,
}
READOUT_PARAMS = {
    "trig": "self",
    "lb": "roi",
    "acq": "raw",
    "ped": "zero",
    "readoutEn": True,
    "singleEv": False,
}
DC.set_window(**READ_WINDOW)
DC.readout_settings = READOUT_PARAMS


CHANNEL_INPUT.rf_on()
time.sleep(0.5)
data = DC.iter(count=10, attempts=3).collect()
CHANNEL_INPUT.rf_off()
print(data)